In [ ]:
import pandas as pd
from DataProcessing import main_data_processing, distance_formula
import plotly.graph_objects as go
outside_assignment_df = pd.read_table('/Volumes/Transcend/fawzi_pycharm_project/fto_assignment_files/Full_length/Run/assigned_peaks_res.out', sep=' +')
outside_peaks_df = pd.read_table('/Volumes/Transcend/fawzi_pycharm_project/fto_assignment_files/Full_length/CS17_FTO', sep=' +', index_col=False)
peak_list, residue_list = main_data_processing()

outside_peaks_df['N_'] = [float(x)+.23 for x in outside_peaks_df['N']]
outside_peaks_df['H_'] = [float(x)+.08 for x in outside_peaks_df['H']]
outside_peaks_df['CO-1'] = [float(x) if x != '-' else '-' for x in outside_peaks_df['CO-1']]

In [ ]:
old_match_list = [[] for x in range(len(outside_peaks_df))]
H_tolerance = .05
N_tolerance = .15
C_tolerance = .1
total = 0
no_match = 0
nh_match = 0
co_discrim = 0
no_co_discrim = 0
ambiguous = 0
for peak in peak_list:
    h_shift = peak.get_data('TROSYHShift')
    n_shift = peak.get_data('TROSYNShift')
    if h_shift is not None:
        total += 1
        ca_shift = peak.get_data('CAShift')
        cb_shift = peak.get_data('CBShift')
        ca_prime_shift = peak.get_data('CAPrimeShift')
        cb_prime_shift = peak.get_data('CBPrimeShift')
        co_prime_shift = peak.get_data('COPrimeShift')
        df = outside_peaks_df[(abs(outside_peaks_df['N_'] - n_shift) < N_tolerance) & (abs(outside_peaks_df['H_'] - h_shift) < H_tolerance)]
        # if there are no matches
        if len(df) == 0:
            # print('no matches for')
            no_match += 1
        # if there is exactly one match
        elif len(df) == 1:
            nh_match += 1
            old_match_list[df.index[0]].append(peak)
        # if it is ambiguous
        elif len(df) > 1:
            # see if co discrimination works
            # checks to see if all the new peak have CO shifts
            if co_prime_shift is not None and len(df[df['CO-1'] == '-']) == 0:
                df = df[(abs(df['CO-1'] - co_prime_shift) < 1)]
                # successful co discrimination case
                if len(df) == 1:
                    co_discrim += 1
                    old_match_list[df.index[0]].append(peak)
                # unsuccessful
                else:
                    ambiguous += 1
                    print('h:', h_shift, 'n:', n_shift, 'ca:', ca_shift, 'ca-1', ca_prime_shift,
                                          'cb:', cb_shift, 'cb-1:', cb_prime_shift, 'co-1:', co_prime_shift)
                    print(df, '\n\n')
            else:
                print('h:', h_shift, 'n:', n_shift, 'ca:', ca_shift, 'ca-1', ca_prime_shift,
                      'cb:', cb_shift, 'cb-1:', cb_prime_shift, 'co-1:', co_prime_shift)
                print(df, '\n\n')
                ambiguous += 1
print('total:', total, 'NH Match:', nh_match,
      'no match:', no_match, 'co discrim:', co_discrim, 'ambiguous:', ambiguous)

outside_peaks_df['old match'] = old_match_list

In [ ]:
a, b, c  = 0, 0, 0
for index in range(len(outside_peaks_df)):
    if len(outside_peaks_df['old match'][index]) == 1:
        peak = outside_peaks_df['old match'][index][0]
        if peak.get_data('CAShift') is not None and outside_peaks_df['CA'][index] == '-':
            # print(outside_peaks_df.iloc[index, :])
            # print(peak.peak_properties)
            # print('\n\n')
            a += 1
        if peak.get_data('CAPrimeShift') is not None and outside_peaks_df['CA-1'][index] == '-':
            # print(outside_peaks_df.iloc[index, :])
            # print(peak.peak_properties)
            # print('\n\n')
            b += 1
        if len(peak.get_data('NearbyHShift')) != 0:
            print(outside_peaks_df.iloc[index, :])
            print(peak.get_data('NearbyHShift'))
            print('\n\n')
            c += 1

(a,b,c)


In [ ]:
d = int()
for index in range(len(outside_peaks_df)):
    if len(outside_peaks_df['old match'][index]) > 1:
        d += 1
        print(outside_peaks_df.loc[index, : ])
        for peak in outside_peaks_df.loc[index, 'old match']:
            print('H:', peak.get_data('TROSYHShift'), 'N:', peak.get_data('TROSYNShift'),
                  'CA:', peak.get_data('CAShift'), 'CA-1:', peak.get_data('CAPrimeShift'),
                  'CO-1:', peak.get_data('COPrimeShift'), 'NOESY', peak.get_data('NOESYHShift'))
        print('\n\n')



In [ ]:
outside_peaks_df